In [1]:
import pickle
import torch
import torchaudio
import time
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write
from audiocraft.data.audio_utils import convert_audio
from audiocraft.modules.conditioners import ConditioningAttributes

objc[85720]: Class AVFFrameReceiver is implemented in both /Users/maxj/miniconda3/envs/audiocraft/lib/libavdevice.58.8.100.dylib (0x114e54798) and /Users/maxj/miniconda3/envs/audiocraft/lib/python3.9/site-packages/av/.dylibs/libavdevice.59.7.100.dylib (0x1349d8778). One of the two will be used. Which one is undefined.
objc[85720]: Class AVFAudioReceiver is implemented in both /Users/maxj/miniconda3/envs/audiocraft/lib/libavdevice.58.8.100.dylib (0x114e547e8) and /Users/maxj/miniconda3/envs/audiocraft/lib/python3.9/site-packages/av/.dylibs/libavdevice.59.7.100.dylib (0x1349d87c8). One of the two will be used. Which one is undefined.


In [2]:
model = MusicGen.get_pretrained('facebook/musicgen-medium')

/Users/maxj/miniconda3/envs/audiocraft/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [3]:
model.lm

LMModel(
  (cfg_dropout): ClassifierFreeGuidanceDropout(p=0.3)
  (att_dropout): AttributeDropout({})
  (condition_provider): ConditioningProvider(
    (conditioners): ModuleDict(
      (description): T5Conditioner(
        (output_proj): Linear(in_features=768, out_features=1536, bias=True)
      )
    )
  )
  (fuser): ConditionFuser()
  (emb): ModuleList(
    (0-3): 4 x ScaledEmbedding(2049, 1536)
  )
  (transformer): StreamingTransformer(
    (layers): ModuleList(
      (0-47): 48 x StreamingTransformerLayer(
        (self_attn): StreamingMultiheadAttention(
          (out_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (linear1): Linear(in_features=1536, out_features=6144, bias=False)
        (dropout): Dropout(p=0.0, inplace=False)
        (linear2): Linear(in_features=6144, out_features=1536, bias=False)
        (norm1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
  

In [4]:
model.compression_model

EncodecModel(
  (encoder): SEANetEncoder(
    (model): Sequential(
      (0): StreamableConv1d(
        (conv): NormConv1d(
          (conv): Conv1d(1, 64, kernel_size=(7,), stride=(1,))
          (norm): Identity()
        )
      )
      (1): SEANetResnetBlock(
        (block): Sequential(
          (0): ELU(alpha=1.0)
          (1): StreamableConv1d(
            (conv): NormConv1d(
              (conv): Conv1d(64, 32, kernel_size=(3,), stride=(1,))
              (norm): Identity()
            )
          )
          (2): ELU(alpha=1.0)
          (3): StreamableConv1d(
            (conv): NormConv1d(
              (conv): Conv1d(32, 64, kernel_size=(1,), stride=(1,))
              (norm): Identity()
            )
          )
        )
        (shortcut): Identity()
      )
      (2): ELU(alpha=1.0)
      (3): StreamableConv1d(
        (conv): NormConv1d(
          (conv): Conv1d(64, 128, kernel_size=(8,), stride=(4,))
          (norm): Identity()
        )
      )
      (4): SEANetRe

In [7]:
def compute_file_embedding(filename):
    return _embed_codes(filename, _load_input(filename))

def _load_input(filename):
    t1 = time.perf_counter()
    wav, sr = torchaudio.load(filename)
    duration = wav.shape[1] / sr
    if duration > 25 * 60:
        raise ValueError(f"track is too long: {duration}s")
    wav = convert_audio(wav, sr, model.sample_rate, model.audio_channels)
    wav = wav.unsqueeze(0)
    t2 = time.perf_counter()
    print(filename, "load", t2 - t1)
    with torch.no_grad():
        encoded, _ = model.compression_model.encode(wav)
    t3 = time.perf_counter()
    print(filename, "encode", t3-t2)
    return encoded
    
def _embed_codes(filename, encoded):
    t1 = time.perf_counter()
    context_limit = model.max_duration * model.frame_rate - 1
    emb = model.lm.compute_sequence_embedding(encoded[0], context_limit)
    t2 = time.perf_counter()
    print(filename, "embed", t2-t1)
    return emb

In [8]:
toks = _load_input("/Users/maxj/Music/Rips/Spotify/AIFF/2fox - We Are - 6Lxi9O5CWRgjrqvWdbn2lT.aiff")

/Users/maxj/Music/Rips/Spotify/AIFF/2fox - We Are - 6Lxi9O5CWRgjrqvWdbn2lT.aiff load 0.2576355419999956
/Users/maxj/Music/Rips/Spotify/AIFF/2fox - We Are - 6Lxi9O5CWRgjrqvWdbn2lT.aiff encode 22.501715417000007


In [9]:
toks

tensor([[[1077, 1273, 1607,  ...,   83,   83,   83],
         [ 845, 1642,   21,  ..., 2044, 2044, 2044],
         [1872,  373,  650,  ..., 2019, 2019, 2019],
         [1518, 1696, 1284,  ..., 1770, 1770, 1770]]])

In [83]:
toks.max()

tensor(2047)

In [9]:
def _load_wav(filename):
    t1 = time.perf_counter()
    wav, sr = torchaudio.load(filename)
    duration = wav.shape[1] / sr
    if duration > 25 * 60:
        raise ValueError(f"track is too long: {duration}s")
    wav = convert_audio(wav, sr, model.sample_rate, model.audio_channels)
    wav = wav.unsqueeze(0)
    t2 = time.perf_counter()
    print(filename, "load", t2 - t1)
    return wav

def _encode(filename, wav):
    t2 = time.perf_counter()
    with torch.no_grad():
        encoded, _ = model.compression_model.encode(wav)
    t3 = time.perf_counter()
    print(filename, "encode", t3-t2)
    return encoded

In [10]:
model.sample_rate

32000

In [11]:
filename = "/Users/maxj/Music/Rips/Spotify/AIFF/2fox - We Are - 6Lxi9O5CWRgjrqvWdbn2lT.aiff"
wav = _load_wav(filename)

/Users/maxj/Music/Rips/Spotify/AIFF/2fox - We Are - 6Lxi9O5CWRgjrqvWdbn2lT.aiff load 0.19886325000000227


In [12]:
wav.shape

torch.Size([1, 1, 8176000])

In [13]:
model.compression_model.encoder

SEANetEncoder(
  (model): Sequential(
    (0): StreamableConv1d(
      (conv): NormConv1d(
        (conv): Conv1d(1, 64, kernel_size=(7,), stride=(1,))
        (norm): Identity()
      )
    )
    (1): SEANetResnetBlock(
      (block): Sequential(
        (0): ELU(alpha=1.0)
        (1): StreamableConv1d(
          (conv): NormConv1d(
            (conv): Conv1d(64, 32, kernel_size=(3,), stride=(1,))
            (norm): Identity()
          )
        )
        (2): ELU(alpha=1.0)
        (3): StreamableConv1d(
          (conv): NormConv1d(
            (conv): Conv1d(32, 64, kernel_size=(1,), stride=(1,))
            (norm): Identity()
          )
        )
      )
      (shortcut): Identity()
    )
    (2): ELU(alpha=1.0)
    (3): StreamableConv1d(
      (conv): NormConv1d(
        (conv): Conv1d(64, 128, kernel_size=(8,), stride=(4,))
        (norm): Identity()
      )
    )
    (4): SEANetResnetBlock(
      (block): Sequential(
        (0): ELU(alpha=1.0)
        (1): StreamableConv1d

In [14]:
model.compression_model.sample_rate

32000

In [15]:
model.compression_model.encoder.model[1].shortcut
# if this is Identity then apparently it's streamable...

Identity()

In [56]:
def _enc(wav):
    t2 = time.perf_counter()
    with torch.no_grad():
        encoded = model.compression_model.encoder(wav)
    t3 = time.perf_counter()
    print("encode", t3-t2)
    return encoded

def _quant(encoded):
    t2 = time.perf_counter()
    with torch.no_grad():
        codes = model.compression_model.quantizer.encode(encoded)
    t3 = time.perf_counter()
    print("quantize", t3-t2)
    return codes

def _quantenc(wav):
    return _quant(_enc(wav))

In [17]:
emb = _enc(wav)

encode 15.903300832999996


In [18]:
emb.shape, emb.dtype

(torch.Size([1, 128, 12775]), torch.float32)

In [19]:
codes = _quant(emb)

quantize 0.11134899999999703


In [20]:
codes.shape, codes.dtype

(torch.Size([1, 4, 12775]), torch.int64)

In [21]:
model.compression_model.encoder.model[0].pad_mode

'reflect'

In [22]:
model.compression_model.causal, model.compression_model.renormalize

(False, False)

In [26]:
enc_m = model.compression_model.encoder
dict(
  channels=enc_m.channels,
  dimension=enc_m.dimension,
  n_filters=enc_m.n_filters,
  ratios=enc_m.ratios,
  n_residual_layers=enc_m.n_residual_layers,
  hop_length=enc_m.hop_length,
  n_blocks=enc_m.n_blocks,
  disable_norm_outer_blocks=enc_m.disable_norm_outer_blocks,
)

{'channels': 1,
 'dimension': 128,
 'n_filters': 64,
 'ratios': [4, 4, 5, 8],
 'n_residual_layers': 1,
 'hop_length': 640,
 'n_blocks': 6,
 'disable_norm_outer_blocks': 0}

In [29]:
def get_wav_sample(n_samples):
    # Calculate the start index for the middle 512 samples
    middle_start = (wav.shape[-1] - n_samples) // 2
    # Extract the middle 512 samples
    return wav[:, :, middle_start:middle_start + n_samples]

In [34]:
wav_sample = get_wav_sample(640*2)
emb = _enc(wav_sample)
emb.shape

encode 0.03221195900005114


torch.Size([1, 128, 2])

In [43]:
emb2 = _enc(torch.cat([wav_sample[:,:,:640],wav_sample[:,:,640:]], dim=0))

encode 0.028904791999934787


In [45]:
emb2.shape

torch.Size([2, 128, 1])

In [58]:
toks1 = _quantenc(wav_sample)

encode 0.02915387500001998
quantize 0.0017911669999648439


In [59]:
toks2 = _quantenc(torch.cat([wav_sample, torch.zeros((1, 1, 640))], dim=2))

encode 0.03327204199990774
quantize 0.0016648750001877488


In [60]:
toks1

tensor([[[1407, 1847],
         [1051,  919],
         [1915,  674],
         [ 656, 1018]]])

In [61]:
toks2

tensor([[[1407, 1249, 1022],
         [1631,  919,  407],
         [ 666,  523, 1646],
         [ 552, 1018, 1966]]])

In [64]:
s = get_wav_sample(640)

In [71]:
import json
json.dumps(s.tolist())

'[[[0.4150869846343994, 0.4161792993545532, 0.44218045473098755, 0.4662434458732605, 0.45270007848739624, 0.44550877809524536, 0.4206930994987488, 0.35829418897628784, 0.3457542657852173, 0.3003555238246918, 0.16770529747009277, 0.07177292555570602, 0.06345430761575699, 0.009831365197896957, -0.05142027884721756, -0.02378156967461109, 0.003404872491955757, -0.019529061391949654, -0.029283417388796806, -0.0017957165837287903, -0.0022508781403303146, 0.0065666548907756805, 0.04451160132884979, 0.052294205874204636, 0.003549434244632721, -0.05355680733919144, -0.1266367882490158, -0.23277875781059265, -0.29489585757255554, -0.2980727553367615, -0.312517911195755, -0.3559313416481018, -0.365296870470047, -0.3110319674015045, -0.2468869835138321, -0.22496899962425232, -0.19663585722446442, -0.1378566473722458, -0.10455043613910675, -0.11567460000514984, -0.12251769006252289, -0.11893727630376816, -0.15665119886398315, -0.17364263534545898, -0.15536093711853027, -0.1658610999584198, -0.17133

In [72]:
filename

'/Users/maxj/Music/Rips/Spotify/AIFF/2fox - We Are - 6Lxi9O5CWRgjrqvWdbn2lT.aiff'

In [73]:
with open(filename, 'rb') as f:
    data = f.read()

In [75]:
import io
wav, sr = torchaudio.load(io.BytesIO(data))

In [77]:
wav.shape

torch.Size([2, 11267550])

In [87]:
import numpy as np

In [92]:
memfile = io.BytesIO()
np.save(memfile, toks.numpy())
serialized = memfile.getvalue()
len(serialized)

408928

In [94]:
memfile = io.BytesIO()
np.save(memfile, toks.numpy().astype(np.int16))
serialized = memfile.getvalue()
len(serialized)

102328

In [95]:
import gzip

memfile = io.BytesIO()
with gzip.GzipFile(fileobj=memfile, mode='wb') as gzfile:
    np.save(gzfile, toks.numpy().astype(np.int16))
serialized = memfile.getvalue()
len(serialized)

71669

In [97]:
wav.shape

torch.Size([2, 11267550])

In [100]:
get_wav_sample(100).shape

IndexError: too many indices for tensor of dimension 2

In [114]:
def batch_indexes(total_len, context_limit, window_size):
    # Tuples (batch_start, batch_end, window_start)
    start = min(context_limit, total_len)
    idxs = [(0, start, 0)]
    for window_start in range(start, total_len, window_size):
        batch_end = window_start + window_size
        if batch_end > total_len:
            batch_end = total_len
        batch_start = batch_end - context_limit
        idxs.append((batch_start, batch_end, window_start))
    return idxs

In [171]:
def get_num_batches(input_len: int, context_limit: int, window_size: int):
    # First batch
    num_batches = 1
    if input_len > context_limit:
        remaining = (input_len - context_limit)
        # Intermediate batches
        num_batches += remaining // window_size
        # Final batch (if necessary)
        if remaining % window_size != 0:
            num_batches += 1
    return num_batches

def get_relevant_embeddings(embed_output: torch.Tensor, window_size: int, input_len: int):
    B, T, dim = embed_output.shape
    B_expected = get_num_batches(input_len, T, window_size)
    assert B_expected == B, f"expected {B_expected} got {B}"

    # Handling the first batch
    relevant_parts = [embed_output[0]]

    # Handling the intermediate batches (if any)
    if B > 2:
        intermediate_parts = embed_output[1:-1, -window_size:]
        relevant_parts.append(intermediate_parts.reshape(-1, dim))

    if B > 1:
        # Handling the last batch
        last_batch_effective_size = input_len - T - (B - 2) * window_size
        last_batch_part = embed_output[-1, -last_batch_effective_size:]
        relevant_parts.append(last_batch_part.reshape(-1, dim))
    
        # Concatenate into a single tensor
        relevant_embeddings = torch.cat(relevant_parts, dim=0)
    else:
        relevant_embeddings = relevant_parts[0]

    assert relevant_embeddings.shape[0] == input_len
    return relevant_embeddings

In [172]:
get_relevant_embeddings(torch.ones(4, 100, 23), 20, 141).mean(dim=0)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1.])

In [169]:
def input_batches(input: torch.Tensor, context_limit: int, window_size: int):
    """ input is of shape (_, T) where T is the input length """
    K, T = input.shape
    num_batches = get_num_batches(T, context_limit, window_size)

    batches = []

    # Generate the batches
    for i in range(num_batches):
        start_index = i * window_size
        end_index = start_index + context_limit
        if end_index > T:
            end_index = T
            start_index = end_index - context_limit
        print(f'{start_index=}, {end_index=}')
        chunk = input[:, start_index:end_index]
        batches.append(chunk)
    assert len(batches) == num_batches

    # Concatenate the batches along the batch dimension
    batched_input = torch.stack(batches, dim=0)

    return batched_input

In [170]:
input_batches(torch.ones((1, 141)), 100, 20)

start_index=0, end_index=100
start_index=20, end_index=120
start_index=40, end_index=140
start_index=41, end_index=141


tensor([[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 

In [219]:
def tensor_chunk(input_tensor, chunk_size, dim=0):
    chunks = []
    input_size = input_tensor.size(dim)
    num_chunks = input_size // chunk_size
    for i in range(num_chunks):
        chunks.append(input_tensor.narrow(dim, i * chunk_size, chunk_size))
    if input_size % chunk_size != 0:
        chunks.append(input_tensor.narrow(dim, num_chunks * chunk_size, input_size % chunk_size))
    return chunks

In [235]:
def empty_condition_tensors(a, b):
    pass
def compute_embeddings(
    lm, condition_tensors, sequence: torch.Tensor
) -> torch.Tensor:
    """
    mock version"""
    B, K, T = sequence.shape
    return torch.ones((B, T, 23))

In [254]:
def get_num_batches(input_len: int, context_limit: int, window_size: int):
    # First batch
    num_batches = 1
    if input_len > context_limit:
        remaining = input_len - context_limit
        # Intermediate batches
        num_batches += remaining // window_size
        # Final batch (if necessary)
        if remaining % window_size != 0:
            num_batches += 1
    return num_batches


def input_batches(input_tensor: torch.Tensor, context_limit: int, window_size: int):
    """input_tensor is of shape (_, T) where T is the input_tensor length"""
    print(f"{input_tensor.shape=}")
    input_len = input_tensor.shape[-1]
    num_batches = get_num_batches(input_len, context_limit, window_size)
    # Generate the batches
    batches = []
    for i in range(num_batches):
        start_index = i * window_size
        end_index = start_index + context_limit
        if end_index > input_len:
            end_index = input_len
            start_index = end_index - context_limit
        chunk = input_tensor[:, start_index:end_index]
        batches.append(chunk)
    assert len(batches) == num_batches
    # Concatenate the batches along the batch dimension
    return torch.stack(batches, dim=0)


def compute_all_embeddings_from_codes(
    lm, codes, context_limit, window_size, max_batch=100
):
    input_len = codes.shape[-1]
    # Turn the input into batches to fit the context limit of the transformer
    batches = input_batches(codes, context_limit, window_size)
    num_batches = batches.shape[0]
    # Chunk the batches to ensure we stay within GPU memory limit
    chunks = tensor_chunk(batches, max_batch)
    num_chunks = len(chunks)
    print(f"{num_batches=} {num_chunks=}")
    chunk_embeddings = []
    for i, batch in enumerate(chunks):
        # Compute embeddings for the chunk
        with torch.no_grad():
            condition_tensors = empty_condition_tensors(lm, batch)
            embeddings = compute_embeddings(lm, condition_tensors, batch)
        B, T, dim = embeddings.shape
        batch_embeddings = []
        # Handle the first batch of the first chunk - it is included in its entirety
        if i == 0:
            batch_embeddings.append(embeddings[0])
        # Handle intermediate batches - only the last window_size embeddings are included
        if num_batches > 2:
            a, b = 0, B
            if i == 0:
                a += 1  # In the first chunk, skip the first batch
            if i == num_chunks - 1:
                b -= 1  # In the last chunk, skip the last batch
            intermediate_embeddings = embeddings[a:b, -window_size:]
            batch_embeddings.append(intermediate_embeddings.reshape(-1, dim))
        # Handling the last batch - output size varies based on input size
        if i == num_chunks - 1 and num_batches > 1:
            last_batch_effective_size = input_len - T - (num_batches - 2) * window_size
            last_batch_embeddings = embeddings[-1, -last_batch_effective_size:]
            batch_embeddings.append(last_batch_embeddings.reshape(-1, dim))
        # Combine all batch embeddings and add to chunk embeddings
        chunk_embeddings.append(torch.cat(batch_embeddings, dim=0))
    # Finally, combine all chunk embeddings
    all_embeddings = torch.cat(chunk_embeddings, dim=0)
    assert all_embeddings.shape[0] == input_len
    return all_embeddings

In [253]:
compute_all_embeddings_from_codes(model.lm, codes[0], 1500, 50)

input_tensor.shape=torch.Size([4, 12775])
num_batches=227 num_chunks=3
last_batch_effective_size=25


tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [220]:
tensor_chunk(torch.tensor([1, 2, 3, 4, 5]), 2)

[tensor([1, 2]), tensor([3, 4]), tensor([5])]

In [10]:
toks[..., :1499].shape

torch.Size([1, 4, 1499])

In [24]:
pattern = model.lm.pattern_provider.get_pattern(1499)

In [25]:
gen_sequence, indexes, mask = pattern.build_pattern_sequence(toks[..., :1499].contiguous(), model.lm.special_token_id)

In [26]:
gen_sequence.shape

torch.Size([1, 4, 1503])

In [27]:
def compute_embeddings(
    lm, condition_tensors, sequence: torch.Tensor
) -> torch.Tensor:
    """This is a copy of the LMModel.forward function, omitting the final layer that maps the
    embeddings to logits.

    Apply the language model on sequence to generate embeddings.
    Given a tensor of sequence of shape [B, K, S] with K the number of codebooks and
    S the sequence steps, return the logits with shape [B, card, K, S].

    Args:
        lm (audiocraft.models.lm.LMModel): The pretrained LMModel to use for embeddings.
        condition_tensors (dict[str, ConditionType], optional): Pre-computed conditioning
            tensors, get via `empty_condition_tensors`.
        sequence (torch.Tensor): Model inputs.
    Returns:
        torch.Tensor: Logits.
    """
    _, K, _ = sequence.shape
    assert (
        K == lm.num_codebooks
    ), "Sequence shape must match the specified number of codebooks"
    input_ = sum([lm.emb[k](sequence[:, k]) for k in range(K)])

    input_, cross_attention_input = lm.fuser(input_, condition_tensors)

    out = lm.transformer(input_, cross_attention_src=cross_attention_input)
    if lm.out_norm:
        out = lm.out_norm(out)
    return out


def empty_condition_tensors(lm, sequence: torch.Tensor):
    """Compute condition tensors suitable for use with the musicgen transformer model, corresponding
    to an empty input for each input in the batch.
    """
    conditions = [ConditioningAttributes(text={"description": None}) for _ in sequence]
    conditions = lm.cfg_dropout(conditions)
    conditions = lm.att_dropout(conditions)
    tokenized = lm.condition_provider.tokenize(conditions)
    # encode conditions and fuse, both have a streaming cache to not recompute when generating.
    condition_tensors = lm.condition_provider(tokenized)
    return condition_tensors

In [28]:
condition_tensors = empty_condition_tensors(model.lm, gen_sequence)
embeddings = compute_embeddings(model.lm, condition_tensors, gen_sequence)
embeddings.shape

torch.Size([1, 1503, 1536])